### Group-by apply

In [48]:
import pandas as pd
import numpy as np

pd.set_option("display.max_rows", 500)

In [3]:
pd_JH_data=pd.read_csv('../data/processed/COVID_relational_confirmed.csv',sep=',',parse_dates=[0])
pd_JH_data=pd_JH_data.sort_values('date',ascending=True).reset_index(drop=True).copy()
pd_JH_data.head()

,date,state,country,confirmed
0,2020-01-22,Alberta,Canada,0.0
1,2020-01-22,no,"Korea, South",1.0
2,2020-01-22,no,Kosovo,0.0
3,2020-01-22,no,Kuwait,0.0
4,2020-01-22,no,Kyrgyzstan,0.0


### Test Data

In [4]:
test_data=pd_JH_data[((pd_JH_data['country']=='US')|
                      (pd_JH_data['country']=='Germany'))&
                     (pd_JH_data['date']>'2020-03-20')]

In [5]:
test_data

,date,state,country,confirmed
15737,2020-03-21,no,Germany,22213.0
15776,2020-03-21,no,US,25818.0
16002,2020-03-22,no,Germany,24873.0
16057,2020-03-22,no,US,33756.0
16268,2020-03-23,no,Germany,29056.0
...,...,...,...,...
41312,2020-06-25,no,US,2422299.0
41538,2020-06-26,no,Germany,194036.0
41578,2020-06-26,no,US,2467554.0
41805,2020-06-27,no,Germany,194458.0


In [6]:
test_data.groupby(['country']).agg(np.max)

,date,state,confirmed
country,,,
Germany,2020-06-27,no,194458.0
US,2020-06-27,no,2510151.0


In [7]:
# %load ../src/features/build_features.py

import numpy as np
from sklearn import linear_model
import pandas as pd

from scipy import signal

reg = linear_model.LinearRegression(fit_intercept=True)

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope

if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

   


the test slope is: [2.]


In [8]:
test_data.groupby(['state','country']).agg(np.max)

date  confirmed
state country                      
no    Germany 2020-06-27   194458.0
      US      2020-06-27  2510151.0

In [9]:
#test_data.groupby(['state', 'country']).apply(get_doubling_time_via_regression)

In [10]:
def rolling_reg(df_input,col='confirmed'):
    ''' input has to be a data frame'''
    ''' return is single series (mandatory for group by apply)'''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)
    return result

In [11]:
test_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed')

state  country       
no     Germany  15737          NaN
                16002          NaN
                16268     7.417994
                16535     7.142035
                16801     8.012983
                           ...    
       US       40782    70.290297
                41047    67.043865
                41312    63.737541
                41578    56.884790
                41846    56.155079
Name: confirmed, Length: 198, dtype: float64

In [12]:
test_data

,date,state,country,confirmed
15737,2020-03-21,no,Germany,22213.0
15776,2020-03-21,no,US,25818.0
16002,2020-03-22,no,Germany,24873.0
16057,2020-03-22,no,US,33756.0
16268,2020-03-23,no,Germany,29056.0
...,...,...,...,...
41312,2020-06-25,no,US,2422299.0
41538,2020-06-26,no,Germany,194036.0
41578,2020-06-26,no,US,2467554.0
41805,2020-06-27,no,Germany,194458.0


In [13]:
pd_DR_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(rolling_reg,'confirmed').reset_index()

In [14]:
pd_DR_result

,state,country,level_2,confirmed
0,Alberta,Canada,0,NaN
1,Alberta,Canada,435,NaN
2,Alberta,Canada,700,NaN
3,Alberta,Canada,967,NaN
4,Alberta,Canada,1263,NaN
...,...,...,...,...
42023,no,Zimbabwe,40762,28.259259
42024,no,Zimbabwe,41029,58.037037
42025,no,Zimbabwe,41294,41.179487
42026,no,Zimbabwe,41560,35.311828


In [15]:
pd_DR_result=pd_DR_result.rename(columns={'confirmed':'confirmed_DR',
                             'level_2':'index'})
pd_DR_result.head()

,state,country,index,confirmed_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,435,NaN
2,Alberta,Canada,700,NaN
3,Alberta,Canada,967,NaN
4,Alberta,Canada,1263,NaN


In [16]:
pd_JH_data=pd_JH_data.reset_index()
pd_JH_data.head()

,index,date,state,country,confirmed
0,0,2020-01-22,Alberta,Canada,0.0
1,1,2020-01-22,no,"Korea, South",1.0
2,2,2020-01-22,no,Kosovo,0.0
3,3,2020-01-22,no,Kuwait,0.0
4,4,2020-01-22,no,Kyrgyzstan,0.0


### confirmed_DR == doubling_rate

In [45]:
pd_result_larg=pd.merge(pd_JH_data,pd_DR_result[['index','confirmed_DR']],on=['index'],how='left')


#pd_result_larg['confirmed_DR'] = pd_result_larg['confirmed_DR'].replace("NaN",int(0))

#pd_result_larg.fillna(0, inplace=True)

pd_result_larg.head()


,index,date,state,country,confirmed,confirmed_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN


## Filtering the data with groupby apply 


In [18]:
from scipy import signal

def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function 
        it ensures that the data structure is kept'''
    window=5, 
    degree=1
    df_result=df_input
    
    filter_in=df_input[column].fillna(0) # attention with the neutral element here
    
    result=signal.savgol_filter(np.array(filter_in),
                           5, # window size used for filtering
                           1)
    df_result[column+'_filtered']=result
    return df_result
        

In [19]:
pd_filtered_result=pd_JH_data[['state','country','confirmed']].groupby(['state','country']).apply(savgol_filter).reset_index()

In [47]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_result[['index','confirmed_filtered']],on=['index'],how='left')
pd_result_larg.tail(100)

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered_x,confirmed_filtered_y
41928,41928,2020-06-27,Hebei,China,349.0,inf,349.2,349.2
41929,41929,2020-06-27,no,Zambia,1531.0,89.392157,1535.0,1535.0
41930,41930,2020-06-27,Alberta,Canada,7957.0,149.031447,7943.4,7943.4
41931,41931,2020-06-27,Anguilla,United Kingdom,3.0,inf,3.0,3.0
41932,41932,2020-06-27,Anhui,China,991.0,inf,991.0,991.0
...,...,...,...,...,...,...,...,...
42023,42023,2020-06-27,no,Barbados,97.0,inf,97.0,97.0
42024,42024,2020-06-27,no,Belarus,61095.0,170.350631,61121.2,61121.2
42025,42025,2020-06-27,no,Belgium,61209.0,605.023102,61207.2,61207.2
42026,42026,2020-06-27,no,Albania,2330.0,32.806763,2334.6,2334.6


## Filtered doubling rate

### doubling_rate_filtered = confirmed_filtered_DR

In [21]:

pd_filtered_doubling=pd_result_larg[['state','country','confirmed_filtered']].groupby(['state','country']).apply(rolling_reg,'confirmed_filtered').reset_index()

pd_filtered_doubling=pd_filtered_doubling.rename(columns={'confirmed_filtered':'confirmed_filtered_DR',
                             'level_2':'index'})

pd_filtered_doubling

,state,country,index,confirmed_filtered_DR
0,Alberta,Canada,0,NaN
1,Alberta,Canada,435,NaN
2,Alberta,Canada,700,NaN
3,Alberta,Canada,967,NaN
4,Alberta,Canada,1263,NaN
...,...,...,...,...
42023,no,Zimbabwe,40762,41.333333
42024,no,Zimbabwe,41029,36.208333
42025,no,Zimbabwe,41294,42.099738
42026,no,Zimbabwe,41560,48.619259


In [22]:
pd_result_larg=pd.merge(pd_result_larg,pd_filtered_doubling[['index','confirmed_filtered_DR']],on=['index'],how='left')
pd_result_larg.tail()

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
42023,42023,2020-06-27,no,Barbados,97.0,inf,97.0,-3.412884e+15
42024,42024,2020-06-27,no,Belarus,61095.0,170.350631,61121.2,1.524167e+02
42025,42025,2020-06-27,no,Belgium,61209.0,605.023102,61207.2,6.074215e+02
42026,42026,2020-06-27,no,Albania,2330.0,32.806763,2334.6,3.138003e+01
42027,42027,2020-06-27,no,Zimbabwe,567.0,69.958333,569.8,4.854783e+01


In [23]:
mask=pd_result_larg['confirmed']>100
pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN) 


In [24]:
pd_result_larg

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered,confirmed_filtered_DR
0,0,2020-01-22,Alberta,Canada,0.0,NaN,0.0,NaN
1,1,2020-01-22,no,"Korea, South",1.0,NaN,0.8,NaN
2,2,2020-01-22,no,Kosovo,0.0,NaN,0.0,NaN
3,3,2020-01-22,no,Kuwait,0.0,NaN,0.0,NaN
4,4,2020-01-22,no,Kyrgyzstan,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...
42023,42023,2020-06-27,no,Barbados,97.0,inf,97.0,NaN
42024,42024,2020-06-27,no,Belarus,61095.0,170.350631,61121.2,152.416667
42025,42025,2020-06-27,no,Belgium,61209.0,605.023102,61207.2,607.421471
42026,42026,2020-06-27,no,Albania,2330.0,32.806763,2334.6,31.380028


In [55]:
pd_result_larg.replace([np.inf, -np.inf], np.nan,inplace=True)

pd_result_larg[pd_result_larg['country']=='United Kingdom'].tail(400)

,index,date,state,country,confirmed,confirmed_DR,confirmed_filtered_x,confirmed_filtered_y
32364,32364,2020-05-22,Channel Islands,United Kingdom,558.0,NaN,558.0,558.0
32369,32369,2020-05-22,Falkland Islands (Malvinas),United Kingdom,13.0,NaN,13.0,13.0
32374,32374,2020-05-22,Gibraltar,United Kingdom,151.0,150.333333,151.4,151.4
32427,32427,2020-05-22,Turks and Caicos Islands,United Kingdom,12.0,NaN,12.0,12.0
32538,32538,2020-05-23,no,United Kingdom,257154.0,81.359483,256600.0,256600.0
32588,32588,2020-05-23,Isle of Man,United Kingdom,336.0,NaN,336.0,336.0
32597,32597,2020-05-23,Montserrat,United Kingdom,11.0,NaN,11.0,11.0
32621,32621,2020-05-23,Anguilla,United Kingdom,3.0,NaN,3.0,3.0
32626,32626,2020-05-23,Bermuda,United Kingdom,128.0,84.666667,129.4,129.4
32629,32629,2020-05-23,British Virgin Islands,United Kingdom,8.0,NaN,8.0,8.0


In [26]:
pd_result_larg.to_csv('../data/processed/COVID_final_set_trial.csv',sep=',',index=False)